# Introduction to python for hydrologists &mdash; pandas

<img src = "./data/pandas/panda.jpg">

## Not that type of Panda -- Python's Pandas package

Pandas is a powerful, flexible and easy to use open source data analysis and manipulation tool. Pandas is commonly used for operations that would normally be done in a spreadsheet environment and includes powerful data analysis and manipulation tools.

Let's begin by importing the libraries and setting our data path 

In [ ]:
import os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import pygeohydro as gh
import datetime 
import statsmodels.api as sm
from matplotlib.backends.backend_pdf import PdfPages
from scipy.signal import detrend

data_path = os.path.join("data", "pandas")

## Get site information data for part of the Russian River near Guerneville, CA from NWIS 

In [ ]:
bbox = (-123.10, 38.45, -122.90, 38.55)
gh.interactive_map(bbox)

Wow that's a lot of gages in this location, let's info on all of them and then use this data to learn about pandas

In [ ]:
query = {"bBox": ",".join(f"{b :.6f}" for b in bbox)}

nwis = gh.NWIS()
info = nwis.get_info(query)
info

The data returned to us from our NWIS query `info` was returned to us as pandas `DataFrame`. We'll be working with this to start learning about the basics of pandas.

## Viewing data in pandas

Pandas has built in methods to inspect `DataFrame` objects. We'll look at a few handy methods:

   - `.head()`: inspect the first few rows of data
   - `.tail()`: inspect the last few rows of data
   - `.index()`: show the row indexes
   - `.columns()`: show the column names
   - `.describe()`: statistically describe the data

In [ ]:
info.head()

In [ ]:
info.tail()

In [ ]:
# print the index names
print(info.index)

In [ ]:
# print the column names
print(info.columns)

# print the column names as a list
print(list(info))

The `describe()` method is only useful for numerical data. This example is does not have a lot of useful data for `describe()`, however we'll use it agian later

In [ ]:
info.describe()

## Getting data from a pandas dataframe

There are multiple methods to get data out of a pandas dataframe as either a "series", numpy array, or a list

Let's start by getting data as a series using a few methods

In [ ]:
# get a series of site numbers by key
info["site_no"]

In [ ]:
# get station names by attribute
info.station_nm

getting data from a dataframe as a numpy array can be accomplished by using `.values`

In [ ]:
info.station_nm.values

### Selection by position

We can get data by position in the dataframe using the `.iloc` attribute 

In [ ]:
info.iloc[0:2, 1:4]

### Selection by label

Pandas allows the user to get data from the dataframe by index and column labels

In [ ]:
info.loc[0:3, ["site_no", "station_nm", "site_tp_cd"]]

### Boolean indexing

pandas dataframes supports boolean indexing that allows a user to create a new dataframe with only the data that meets a boolean condition defined by the user.

Let's get a dataframe of only groundwater sites from the `info` dataframe

In [ ]:
dfgw = info[info["site_tp_cd"] == "GW"]
dfgw

### Reading and writing data to `.csv` files

Pandas has support to both read and write many types of files. For this example we are focusing on `.csv` files. For information on other file types that are supported see the [ten minutes to pandas](https://pandas.pydata.org/docs/user_guide/10min.html#importing-and-exporting-data) tutorial documentation

For this part we'll write a new `.csv` file of the groundwater sites that we found in NWIS using `to_csv()`.

`to_csv()` has a bunch of handy options for writing to file. For this example, I'm going to drop the index column while writing by passing `index=False`

In [ ]:
csv_file = os.path.join(data_path, "RussianRiverGWsites.csv")
dfgw.to_csv(csv_file, index=False)

Now we can load the csv file back into a pandas dataframe with the `read_csv()` method.

In [ ]:
dfgw2 = pd.read_csv(csv_file)
dfgw2

## Class exercise 1 

Using the methods presented in this notebook create a `DataFrame` of surface water sites from the `info` dataframe, write it to a csv file named `"RussianRiverSWsites.csv"`, and read it back in as a new `DataFrame`

## Get timeseries data from NWIS for surface water sites

Now to start working with stream gage data from NWIS. We're going to send all surface water sites to NWIS and get only sites with daily discharge measurements.

([11467002](https://waterdata.usgs.gov/nwis/inventory?agency_code=USGS&site_no=11467002))

In [ ]:
dfsw = info[info.site_tp_cd == "ST"]
dates = ("1939-09-22", "2022-09-01")
stations = dfsw.site_no.to_list()
df = nwis.get_streamflow(stations, dates)
df.head()

Awesome! There are two gages in this study area that have daily discharge data!

### Inspect the data using `.plot()`

In [ ]:
ax = df.plot()
ax.set_ylabel(r"cubic meter per second")
ax.set_yscale("log")

### Updating column names

Let's update column names for these gages based on their locations instead of their USGS gage id. First we'll get name information from the `info` class and then we'll use the station name information to remap the column names.

the `rename()` method accepts a dictionary that is formated `{current_col_name: new_col_name}`

In [ ]:
remap = {}
for i in list(df):
    site_no = i.split("-")[-1]
    desc = info[info.site_no == site_no]["station_nm"].values[0]
    colname = "_".join(desc.lower().split()[:2])
    remap[i] = colname
    
df2 = df.rename(columns=remap)
df2.plot();

### Adding a new column of data to the dataframe

Adding new columns of data is relatively simple in pandas. The call signature is similar to a dictionary where df["key"] = "some_array_of_values"

Let's add a year column (water year) to the dataframe by getting the year from the index and adjusting it using `np.where()`

In [ ]:
df2['year'] = np.where(df2.index.month >=10, df2.index.year, df2.index.year - 1)

### Manipulating data

Columns in a pandas dataframe can be manipulated similarly to working with a dictionary of numpy arrays. The discharge data units are $\frac{m^3}{s}$, let's accumulate this to daily discharge in $\frac {ft^3}{d}$.

The coversion for this is:

## $ \frac{1 m^3}{s} \times \frac{35.3147 ft^3}{1 m^3} \times \frac{60s}{min} \times \frac{60min}{hour} \times \frac{24hours}{day} \rightarrow \frac{ft^3}{day} $

and then convert that to acre-feet (1 acre-ft = 43559.9)

In [ ]:
conv = 35.3147 * 60 * 60 * 24
df2["russian_r_cfd"] = df2.russian_r * conv
df2["russian_r_af"] = df2.russian_r_cfd / 43559.9
df2

# now let's plot up discharge from 2018 - 2020
dfplt = df2[(df2["year"] >= 2015) & (df2["year"] <= 2020)]
ax = dfplt["russian_r_af"].plot()
ax.set_yscale("log")


## Class exercise 2:

Using the methods described in the notebook. Convert the discharge for `"austin_c"` from $\frac{m^3}{s}$ acre-ft by adding two additional fields to `df2` named `"austin_c_cfs"` and `"austin_c_af"`. 

After these two fields have been added to df2 use boolean indexing to create a new dataframe from 2015 through 2020. Finally plot the discharge data (in acre-ft) for "austin_cr".

**bonus exercise** try to plot both `"russian_r_af"` and `"austin_c_af"` on the same plot

### groupby: grouping data and performing mathematical operations on it

`groupby()` is a powerful method that allows for performing statistical operations over a groups of "common" data within a dataframe. 

For this example we'll use it to get mean daily flows for the watershed. Pandas will group all common days of the year with each other and then calculate the mean value of these via the function `.mean()`. `groupby()` also supports other operations such as `.median()`, `.sum()`, `max()`, `min()`, `std()`, and other functions. 

In [ ]:
df2["day_of_year"] = df2.index.day_of_year
df2["austin_c_cfd"] = df2.austin_c * conv
df2["austin_c_af"] = df2.austin_c_cfd / 43559.9
df_mean_day = df2.groupby(by=["day_of_year"], as_index=False)[["russian_r_af", "austin_c_af"]].mean()
ax = df_mean_day[["russian_r_af", "austin_c_af"]].plot()
ax.set_yscale("log")

We can see that around March flow starts decreasing heavily and doesn't recover until sometime in october. What's going on here? Let's load some climate data and see what's happening.

In [ ]:
cimis_file = os.path.join(data_path, "santa_rosa_CIMIS_83.csv")
df_cimis = pd.read_csv(cimis_file)
drop_list = ["qc"] + [f"qc.{i}" for i in range(1, 22)]
df_cimis.drop(columns=drop_list, inplace=True)
df_cimis

In [ ]:
df_mean_clim = df_cimis.groupby(by=["Jul"], as_index=False)[["ETo (in)", "Precip (in)"]].mean()
df_mean_clim = df_mean_clim.rename(columns={"Jul": "day_of_year"})
df_mean_clim.head()

### Joining two `DataFrames` with an inner join

Inner joins can be made in pandas with the `merge()` method. As inner join, joins two dataframes on a common key or values, when a key or value exists in one dataframe, but not the other that row of data will be excluded from the joined dataframe. 

There are a number of other ways to join dataframes in pandas. Detailed examples and discussion of the different merging methods can be found [here](https://realpython.com/pandas-merge-join-and-concat/)

In [ ]:
df_merged = pd.merge(df_mean_day, df_mean_clim, on=["day_of_year",])
df_merged.head()

In [ ]:
cols = [i for i in list(df_merged) if i != "day_of_year"]
ax = df_merged[cols].plot()
ax.set_yscale("log")

This starts to make a little more sense! We can see that this area gets most of it's precipitation in the winter and early spring. The evapotranspiration curve shows what we'd expect: that there is more evapotranspiration in the summer than the winter. 

Is there anything else we can look at that might give us more insights into this basin? Let's try looking at long term, yearly trends to see what's there.

In [ ]:
year = []
for dstr in df_cimis.Date.values:
    mo, d, yr = [int(i) for i in dstr.split("/")]
    if mo < 10:
        year.append(yr - 1)
    else:
        year.append(yr)
df_cimis["year"] = year
df_cimis.head()

In [ ]:
df2.head()

### `.aggregate()`

The pandas `.aggregate()` can be used with `.groupy()` to perform multiple statistical operations.

Example useage could be:

```python
    
    agdf = df2.groupby(by=["day_of_year"], as_index=False)["austin_c_af"].aggregate([np.min, np.max, np.mean, np.std])
```

And would return a dataframe grouped by the day of year and take the min, max, mean, and standard deviation of these data.

## Class exercise 3: groupy, aggregate, and merge

For this exercise we need to produce yearly mean and standard deviation flows for the `russian_r_af` variable and merge the data with yearly mean precipitation and evapotranspiration  

**Hints:**  
   - `df2` and `df_cimis` are the dataframes these operations should be performed on  
   - use `.groupby()` to group by the year and do `precip` and `ETo` in seperate groupby routines
   - rename the columns in the ETo aggregated dataframe and the Precip dataframe to `mean_et`, `std_et`, `mean_precip`, `std_precip`
   - `np.mean` and `np.std` can be used to calculate mean and std flows  
   
Name your final joined dataframe `df_yearly`

Lets examine the long term flow record for the Russian River

In [ ]:
df_y_flow = df2.groupby(by=["year"], as_index=False)["russian_r_af"].aggregate([np.mean, np.std])

fig = plt.figure(figsize=(12,4))

lower_ci = df_y_flow["mean"] - 2 * df_y_flow['std']
lower_ci = np.where(lower_ci < 0, 0, lower_ci)
upper_ci = df_y_flow["mean"] + 2 * df_y_flow['std']
ax = df_y_flow["mean"].plot(style="b.-")
ax.fill_between(df_y_flow.index, lower_ci, upper_ci, color="b", alpha=0.5);

From this plot it looks like the long term trend in yearly discharge in the Russian River has been decreasing. We will revisit and test this later. 

First let's see if there are any relationships between yearly discharge and climate

In [ ]:
dfg0 = df2.groupby(by=["year"], as_index=False)["russian_r_af"].aggregate([np.mean, np.std, np.sum])
dfg1 = df_cimis.groupby(by=["year"], as_index=False)["ETo (in)"].aggregate([np.mean, np.std, np.sum])
dfg2 = df_cimis.groupby(by=["year"], as_index=False)["Precip (in)"].aggregate([np.mean, np.std, np.sum])
dfm1 = pd.merge(dfg0, dfg1, on=["year"], suffixes=(None, "_et"))
df_yearly = pd.merge(dfm1, dfg2, on=["year"], suffixes=(None, "_precip"))

fig = plt.figure(figsize=(12,4))

lower_ci = df_yearly["mean"] - 2 * df_yearly['std']
lower_ci = np.where(lower_ci < 0, 0, lower_ci)
upper_ci = df_yearly["mean"] + 2 * df_yearly['std']
ax = df_yearly["mean"].plot(style="b.-", label="flow_af")
ax.fill_between(df_yearly.index, lower_ci, upper_ci, color="b", alpha=0.5)
ax2 = ax.twinx()
ax2.plot(df_yearly.index, df_yearly.sum_precip, "k--", lw=1.5, label="Precip")
ax2.plot(df_yearly.index, df_yearly.sum_et, "r.--", lw=1.5, label="ET")
plt.legend()
plt.show();

As expected, precipitation amount is the main driver of the yearly discharge regime here

### Baseflow separation

Baseflow separation is a method to separate the quick response hydrograph (storm runoff) from the long term flow. We're going to go back to the complete daily dataset to perform this operation. The following cell contains a low pass filtration method that is used for digital baseflow separation. We'll use these in our analysis

In [ ]:
def _baseflow_low_pass_filter(arr, beta, enforce):
    """
    Private method to apply digital baseflow separation filter
    (Lyne & Hollick, 1979; Nathan & Mcmahon, 1990;
    Boughton, 1993; Chapman & Maxwell, 1996).

    This method should not be called by the user!

    Method removes "spikes" which would be consistent with storm flow
    events from data.

    Parameters
    ----------
    arr : np.array
        streamflow or municipal pumping time series

    beta : float
        baseflow filtering parameter that ranges from 0 - 1
        values in 0.8 - 0.95 range used in literature for
        streamflow baseflow separation

    enforce : bool
        enforce physical constraint of baseflow less than measured flow

    Returns
    -------
        np.array of filtered data
    """
    # prepend 10 records to data for initial spin up
    # these records will be dropped before returning data to user
    qt = np.zeros((arr.size + 10,), dtype=float)
    qt[0:10] = arr[0:10]
    qt[10:] = arr[:]

    qdt = np.zeros((arr.size + 10,), dtype=float)
    qbf = np.zeros((arr.size + 10,), dtype=float)

    y = (1. + beta) / 2.

    for ix in range(qdt.size):
        if ix == 0:
            qbf[ix] = qt[ix]
            continue

        x = beta * qdt[ix - 1]
        z = qt[ix] - qt[ix - 1]
        qdt[ix] = x + (y * z)

        qb = qt[ix] - qdt[ix]
        if enforce:
            if qb > qt[ix]:
                qbf[ix] = qt[ix]
            else:
                qbf[ix] = qb

        else:
            qbf[ix] = qb

    return qbf[10:]


def baseflow_low_pass_filter(arr, beta=0.9, T=1, enforce=True):
    """
    User method to apply digital baseflow separation filter
    (Lyne & Hollick, 1979; Nathan & Mcmahon, 1990;
    Boughton, 1993; Chapman & Maxwell, 1996).

    Method removes "spikes" which would be consistent with storm flow
    events from data.

    Parameters
    ----------
    arr : np.array
        streamflow or municipal pumping time series

    beta : float
        baseflow filtering parameter that ranges from 0 - 1
        values in 0.8 - 0.95 range used in literature for
        streamflow baseflow separation

    T : int
        number of recursive filtering passes to apply to the data

    enforce : bool
        enforce physical constraint of baseflow less than measured flow

    Returns
    -------
        np.array of baseflow
    """
    for _ in range(T):
        arr = _baseflow_low_pass_filter(arr, beta, enforce)

    return arr


## Class exercise 4: baseflow separation

Using the full dataframe discharge dataframe, `df2`, get the baseflows for the Russian River (in acre-ft) and add these to the dataframe as a new column named `russian_bf_af`.

The function `baseflow_low_pass_filter()` will be used to perform baseflow. This function takes a numpy array of stream discharge and runs a digital low pass filtration method to calculate baseflow. 

After performing baseflow seperation plot both the baseflow and the total discharge for the years 2015 - 2017.

**Hints:**
   - make sure to use the `.values` property when you get discharge data from the pandas dataframe
   - feel free to play with the input parameters `beta` and `T`. 
      - `beta` is commonly between 0.8 - 0.95 for hydrologic problems.
      - `T` is the number of filter passes over the data, therefore increasing `T` will create a smoother profile. I recommend starting with a `T` value of around 5 and adjusting it to see how it changes the baseflow profile.

## Looking at trends and seasonal cycles in the data with `statsmodels`

`statsmodels` is a Python module that provides classes and functions for the estimation of many different statistical models, as well as for conducting statistical tests, and statistical data exploration. For more information on `statsmodels` and it's many features visit [the getting started guide](https://www.statsmodels.org/dev/gettingstarted.html)

For this example we'll be using time series analysis to perform a seasonal decomposition using moving averages. The `statsmodels` function `seasonal_decompose` will perform this analysis and return the data trend, seasonal fluctuations, and residuals.

In [ ]:
df2["russian_bf_af"] = baseflow_low_pass_filter(df2["russian_r_af"].values, beta=0.9, T=5)
# drop nan values prior to running decomposition
dfsm = df2[df2['russian_bf_af'].notna()]

dfsm.head()
# decompose
decomposition = sm.tsa.seasonal_decompose(dfsm["russian_bf_af"], period=365, model="additive")
 
decomposition.plot()
mpl.rcParams['figure.figsize'] = [14, 14];

Now let's plot our baseflow trend against the baseflow we calculated 

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))

ax = df2["russian_r_af"].plot(ax=ax)
ax.plot(decomposition.trend.index, decomposition.trend, "r-", lw=1.5)
ax.set_yscale('log')
plt.savefig("russian_river.png")


What's going on at the end of the 1970's? Why is the baseflow trend so low and peak flows are also low? 

From ["California's Most Significant Droughts"](https://cawaterlibrary.net/wp-content/uploads/2017/05/CalSignficantDroughts_v10_int.pdf), Chapter 3 

The setting for the 1976-77 drought differed significantly from the dry times of the 1920s-30s. Although
only a two-year event, its hydrology was severe.
Based on 114 years of computed statewide runoff,
1977 occupies rank 114 (driest year) and 1976 is in
rank 104. The drought was notable for the impacts
experienced by water agencies that were unprepared
for such conditions.  One reason for the lack of
preparedness was the perception of relatively ample
water supplies in most areas of the state. 

We can also see the analysis that there is an overall reduction in baseflow from 1940's to the 2020's

### Writing yearly plots of data to a PDF document

We can write the plots we make with pandas to a multipage pdf using matplotlib's `PdfPages`. For this example we'll make yearly plots of all of the discharge, baseflow, and baseflow trend and bundle them into a single pdf document.

In [ ]:
# add the decomposition trend to the dataframe to make plotting easy
df2["baseflow_trend"] = decomposition.trend

pdf_file = os.path.join(data_path, "all_years_russian.pdf")
with PdfPages(pdf_file) as outpdf:
    for year in sorted(df2.year.unique()):
        plt.figure()
        tdf = df2[df2.year == year]
        tdf[["russian_r_af", "russian_bf_af", "baseflow_trend"]].plot()
        plt.xlabel("date")
        plt.ylabel("acre-feet")
        plt.ylim([1, 300000])
        plt.yscale("log")
        plt.tight_layout()
        outpdf.savefig()
        plt.close('all')

Go into the "data/pandas" and open up the file "all_years_russian.pdf" to check out the plots.

## Back to the basics:

Now that we've done some analysis on real data, let's go back to the basic and briefly discuss how to adjust data values in pandas


### Adjusting a whole column of data values

Because pandas series objects (columns) are built off of numpy, we can perform mathematical operations in place on a whole column similarly to numpy arrays


In [ ]:
df3 = df2.copy()
# Adding values
df3["russian_r"] += 10000

# Subtracting values
df3["russian_r"] -= 10000

# multiplication
df3["russian_r"] *= 5

# division
df3["russian_r"] /= 5

# more complex operations
df3["russian_r"] = np.log10(df3["russian_r"].values)
df3.head()

### updating values by position

We can update single or multiple values by their position in the dataframe using `.iloc`

In [ ]:
df3.iloc[0, 1] = 999
df3.head()

### updating values based on location

We can update values in the dataframe based on their index and column headers too using `.loc`

In [ ]:
df3.loc[df3.index[0], 'russian_r'] *= 100
df3.head()

## Class exercise 5: setting data

The russian river discharge in cubic feet per day highly variable and represented by large numbers. To make this data easier to read and plot, update it to a log10 representation.

Then use either a location based method to change the value in austin_c on 10-4-1939 to 0.

Finally display the first 5 records in the notebook.

## Now for some powerful analysis using fourier analysis to extract the data's signal.

Some background for your evening viewing: https://youtu.be/spUNpyF58BY

We have to detrend the data for fast fourier transforms to work properly. Here's a discussion on why: 

https://groups.google.com/forum/#!topic/comp.dsp/kYDZqetr_TU

Fortunately we can easily do this in python using scipy!

https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.detrend.html

Let's create a new dataframe with only the data we're interested in analysing.

In [ ]:
df_data = df2[["russian_r_af", "year"]]
df_data = df_data.rename(columns={"russian_r_af": "Q"})
df_data.describe()

#### Now we're going to detrend the data

But first let's also set the water year by date on this dataframe and drop the single nan value from the dataframe

In [ ]:
df_data['water_year'] = df_data.index.shift(30+31+31,freq='d').year
df_data.dropna(inplace=True)
df_data["detrended"] = detrend(df_data.Q.values)

In [ ]:
fig = plt.figure(figsize=(12,6))
ax = plt.subplot(3,1,1)
plt.plot(df_data.detrended)
plt.title('detrended signal')

plt.subplot(3,1,2)
plt.plot(df_data.index, df_data.Q)
plt.title('Raw Signal')

plt.subplot(3,1,3)
plt.plot(df_data.index, df_data.Q - df_data.detrended)
plt.title('Difference')

plt.tight_layout()

### Evaluate and plot the Period Spectrum to see timing of recurrence

Here we've created a function that performs fast fourier transforms and then plot's the spectrum for signals of various lengths.

In [ ]:
def fft_and_plot(df, plot_dominant_periods=4):
    N = len(df)
    yf = np.fft.fft(df.detrended)
    yf = np.abs(yf[:int(N / 2)])
    
    # get the right frequency 
    # https://docs.scipy.org/doc/numpy/reference/generated/numpy.fft.fftfreq.html#numpy.fft.fftfreq
    d = 1. # day
    f = np.fft.fftfreq(N, d)[:int(N/2)]
    f[0] = .00001
    per = 1./f # days
    
    fig = plt.figure(figsize=(12,6))
    ax = plt.subplot(2,1,1)
    plt.plot(per, yf)
    plt.xscale('log')

    top=np.argsort(yf)[-plot_dominant_periods:]
    j=(10-plot_dominant_periods)/10
    for i in top:
        plt.plot([per[i], per[i]], [0,np.max(yf)], 'r:')
        plt.text(per[i], j*np.max(yf), f"{per[i] :.2f}")
        j+=0.1

    plt.title('Period Spectrum')
    plt.grid()
    ax.set_xlabel('Period (days)')
    plt.xlim([1, 1e4])

    plt.subplot(2,1,2)
    plt.plot(df.index,df.Q)
    plt.title('Raw Signal')
    plt.tight_layout()

### Now let's look at the whole signal

In [ ]:
fft_and_plot(df_data)

We see a strong annual frequency that corresponds to spring snowmelt from the surrounding mountain ranges. The second strongest frequency is biannually, and is likely due to the onset of fall in winter rains

### Okay, what about years before dams were installed on the Russian River.

Let's get in the wayback machine and look at only the years prior to 1954!

In [ ]:
fft_and_plot(df_data[df_data.index.year < 1954])

The binanual and annual signals have a much wider spread, which suggests that peak runoff was a little more variable in it's timing compared to the entire data record.

### What if we focus in on after 1954?

In [ ]:
fft_and_plot(df_data[(df_data.index.year > 1954)])

It looks like flows are more controlled with a tighter periods.

So post dam construction, flows are much more controlled with regular release schedules. 